# Dummy Net

**What will happens when:**
 - second layer will be initialised with 0
 - learning_rate will change to 0.1

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from utils.data import init_dir
import tensorflow as tf
import numpy as np
import os

## Build Net Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('dummy_net_inputs'):
        data = tf.placeholder(tf.float32, shape=[None, 3], name='data')
        labels = tf.placeholder(tf.float32, shape=[None, 1], name='labels')
    
    with tf.variable_scope('layer_1'):
        w1 = tf.get_variable('w1', initializer=tf.constant_initializer(0), shape=[3,2])
        prediction = tf.matmul(data, w1)
        tf.summary.histogram('w1_histogram', w1)
    
    with tf.variable_scope('layer_2'):
        w2 = tf.get_variable('w2', initializer=tf.truncated_normal(shape=[2,1], stddev=0.1))
        prediction = tf.matmul(prediction, w2)
        tf.summary.histogram('w2_histogram', w2)
    
    with tf.name_scope('loss'):
        loss = tf.losses.mean_squared_error(labels, prediction)
        tf.summary.scalar('loss_summary', loss)

    with tf.name_scope('training'):
        train_step = tf.train.GradientDescentOptimizer(learning_rate=0.005).minimize(loss)

    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

In [ ]:
graph.get_all_collection_keys()

## Init Model Logging

In [ ]:
from utils.data import init_model_logging
base_dir = '/tensorboard_summaries/dummy_net/'

logging_meta = init_model_logging(base_dir, 'experiment1', graph=graph, remove_existing=True)

In [ ]:
logging_meta

## Run net

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True

_data = np.array([[1,2,3]])
_labels = np.array([[5]])

with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    
    for iteration in range(100):
        # Prepare data
        feed_dict = {data: _data, labels: _labels}
        fetches = [train_step, prediction, loss, merge_summaries]

        # Train
        _, _prediction, _loss, _summary = session.run(fetches, feed_dict)
        
        # Log
        logging_meta['train_writer'].add_summary(_summary, iteration)
        print("Iteration {}: loss {}, pred {}".format(iteration, _loss, _prediction[0][0]))

    logging_meta['saver'].save(session, logging_meta['model_path'])